---
title: "Workshop Exercise: Grouping, Summarizing and Plotting"
---


by: Marlon Pius Pamplona

## Data Preparation

### Load Packages

To get started, **load in the needed packages**: pandas, numpy, plotly express.


In [ ]:
# Your code here
import pandas as pd
import numpy as np
import plotly.express as px

### Import Data

Download the `india_tuberculosis.csv` file from the course portal and save it in your `data` folder.

Now, **read in the `india_tuberculosis` dataset into Python from your data folder** into a DataFrame called `tb`. The DataFrame you import should have 880 rows and 21 columns. Remember to use relative paths to allow your script to use project-relative paths.


In [ ]:
# Import the CSV file
# Your code here
tb = pd.read_csv("data/india_tuberculosis.csv")

tb

Run the following line of code to convert all column names to lowercase and replace spaces with underscores:


In [ ]:
tb.columns = tb.columns.str.lower().str.replace(' ', '_')

### Calculate Total Cost

Each patient record notes the location and cost of up to three visits. Add together these costs (`first_visit_cost`, `second_visit_cost`, and `third_visit_cost`) to create a `total_cost` variable.


In [ ]:
# Calculate total cost
tb['total_cost'] = tb['first_visit_cost'] + tb['second_visit_cost'] + tb['third_visit_cost']

## Investigating Healthcare Costs by Education Level

### Reorder Education Levels

Before moving further, let's take a closer look at the `education` variable in our dataset:


In [ ]:
# Count unique values of education levels
tb["education"].value_counts()

You'll notice that education levels are ordered in descending order by default. However, in this case, we want to reorder the education levels so that they are arranged in a chronological order (lower education levels come first).

First, use the `replace()` method to convert the "Graduate & Above" values to "Graduate".


In [ ]:
tb["education"] = tb['education'].replace({"Graduate & Above":"Graduate"})

Now, run the code below to reorder the education levels by converting the `education` variable into a categorical variable with the levels defined in a specific order:


In [ ]:
education_levels = [
    "No Education",
    "Primary",
    "Middle",
    "Secondary",
    "Higher Secondary",
    "Graduate",
    "Missing",
]

tb["education"] = pd.Categorical(
    tb["education"], categories=education_levels, ordered=True
)

Now, inspect the `education` variable again to make sure it has been reordered correctly:


In [ ]:
tb["education"].value_counts()

We will use the newly transformed `education` variable for grouping, summarizing, and plotting below.

### Table of Costs by Education Level

Let's compare the cost of treatment at different health facilities using a summary table. Use pandas methods to group by education level and create a summary table with two variables:

- The number of patients in each education level
- The mean total cost for each education level

Store the resulting DataFrame as an object called `edu_cost_table`.


In [ ]:
# Create summary table
edu_cost_table = tb.groupby('education').agg(num_patients=('education','size'), mean_total_cost=('total_cost','mean'))

edu_cost_table

Reorder the rows of the summary table to go from highest to lowest mean cost. You can use the `sort_values()` method to do this.


In [ ]:
# Reorder summary table
edu_cost_table = tb.groupby('education').agg(num_patients=('education','size'), mean_total_cost=('total_cost','mean')).sort_values(by='mean_total_cost',ascending=False).reset_index()

edu_cost_table

Which education level has the highest mean total cost of treatment?

YOUR ANSWER HERE: Graduate

Below, we will visualize the data with a plot to see if there is a relationship between education level and healthcare costs.

### Visualize Costs by Education Level

Using `plotly.express` and the costs summary table you created earlier, create a bar plot of mean costs by education level. It should be sorted from lowest education level to highest education level.


In [ ]:
fig = px.bar(
    edu_cost_table, 
    x='education', 
    y='mean_total_cost', 
    title='Mean Total Cost by Education Level',
    labels={'education': 'Education Level', 'mean_total_cost': 'Mean Total Cost'},
    text_auto=True
)

fig.show()

Write a short interpretation of the plot.

This bar chart visualizes the mean total cost by education level, showing that individuals with a "Graduate" education level have the highest mean cost, while those with "No Education" have the lowest

## Investigating Healthcare Costs by Age Group

### Create an `age_group` Variable

The `age` variable records the age of each patient in years. We want to classify the patients into four approximately equally-sized age groups for further analysis.

First, run the code below to determine the quantiles of the `age` variable to decide the cutoffs for each age group:


In [ ]:
# Determine quantiles of age
tb["age"].describe()

Based on the quantile markers (min, 25%, 50%, 75%, max), we can note the following four age groups with approximately equal numbers of patients:

- 18-36
- 37-47
- 48-57
- 58+

Use a custom function called `assign_age_group()`, then vectorize it using `np.vectorize()` and apply it to the `age` variable to create an `age_group` variable.

(If you prefer, you can also use `pd.cut()` to create the `age_group` variable.)


In [ ]:
# Your code here

#define function
def assign_age_group(age):
    if 18 <= age <=36:
        return "18-36"
    elif 37 <= age <=47:
        return "37-47"
    elif 48 <= age <=57:
        return "48-57" 
    elif age >= 58:
        return "58+"
    else:
        return "Unknown" 
#vectorize
assign_age_group = np.vectorize(assign_age_group)

#tranform data
tb['age_group'] = assign_age_group(tb['age'])

Now, check if we met our goal of having a similar number of patients in each age group using the `value_counts()` method:


In [ ]:
# Tabulate age groups
tb["age_group"].value_counts(normalize=True)

**Checkpoint:** If you classified the age groups correctly, you will see that each age group has approximately 24-26% of the patients.

### Table of Costs by Age Group

Next, summarize the total healthcare costs by age group. Use pandas methods to group by `age_group` and 
create a summary table with two variables:

- The number of patients in each age group
- The mean total cost for each age group

Store the resulting DataFrame as an object called `age_cost_table`.


In [ ]:
# Create summary table for age groups
age_cost_table = tb.groupby('age_group').agg(num_patients=('age_group','size'), mean_total_cost=('total_cost','mean')).sort_values(by='mean_total_cost',ascending=False).reset_index()

age_cost_table

### Visualize Costs by Age Group

Using `plotly.express` and the costs summary table you created earlier, create a bar plot of mean costs by age group.


In [ ]:
fig2 = px.bar(
    age_cost_table, 
    x='age_group', 
    y='mean_total_cost', 
    title='Mean Total Cost by Age Group',
    labels={'age_group': 'Age Group', 'mean_total_cost': 'Mean Total Cost'},
    text_auto=True
)

fig2.show()

Write a short interpretation of the plot.

This bar chart visualizes the mean total cost by age group, showing that individuals with ages 18-36 have the highest mean cost, while those with 48-57 have the lowest

## Bonus Challenge (Optional and Ungraded)

If you finish early and have extra time, you can explore and transform the dataset further.

We can look at two types of health facilities in the dataset: private and public (government).


In [ ]:
# List of healthcare locations
tb["first_visit_location"].unique()

Note: PHC = primary health center, GH = govt hospital, etc.

### Comparing Delays in Public vs Private Health Facilities

Let's create a new variable indicating whether a healthcare location is private or public. We can do this using the `str.contains()` function from pandas.


In [ ]:
tb['first_visit_type'] = np.where(
    tb['first_visit_location'].str.contains('Pvt'),
    'Private',
    'Public'
)

Inspect the new variable in `tb`.


In [ ]:
# Inspect new variable
tb["first_visit_type"].value_counts(normalize=True)

Your assignment for this challenge is to summarize delays in treatment and compare the length of delays between private and public health facilities.

You can start by making a summary table of `treatment_initiation_delay_in_days`, grouped by `first_visit_type`. Then visualize the data and draw insights from your findings.


In [ ]:
# Create summary table for age groups
delay_table = tb.groupby('first_visit_type').agg(delays=('treatment_initiation_delay_in_days','mean')).sort_values(by='delays',ascending=False).reset_index()

delay_table

## Wrap Up

That's it for this assignment! Feel free to explore further and share your findings.